In [48]:
# importing required packages

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import IPython
import csv
import pandas as pd

import matplotlib #scientific plots
import matplotlib.pyplot as plt

import numpy as np # numerical programming

import sklearn #scikit-learn
import math #Mathematical functions
import pylab 
import time
import pprint as pp
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import NullFormatter #No labels on the ticks.
from IPython.display import Image
from __future__ import division

Now we will import the data. Here I am using pandas to import data and I will aslo display some data.

In [49]:
#number of rows and columns to display
#pd.options.display.max_rows = 400
#pd.options.display.max_columns = 30
#reading data from csv file using pandas
collected_data = pd.read_csv('SFE Data.csv', delimiter =',',index_col = None, header = 0)
#collected_data.shape #to check the size of data
collected_data.head(2)



,C,N,P,S,V,Ni,Nb,Al,Ti,Fe,Hf,Mo,Mn,Co,Si,Cr,Cu,DOI,url,SFE,Error,title,Author,temperature,experiment_type
0,0.004,0.003,0.025,0.001,0.0,15.6,0.0,0.0,0.0,64.317,0.0,2.5,0.03,0.0,0.02,17.5,0.0,10.1007/s11661-013-1943-0,http://link.springer.com/article/10.1007/s1166...,51.6,0.014341,The Effect of Chemical Composition and Heat Tr...,"Yonezawa, Toshio",300,TEM - WBDF
1,0.020,0.009,0.022,0.001,0.0,15.6,0.0,0.0,0.0,64.188,0.0,2.5,0.03,0.0,0.03,17.6,0.0,10.1007/s11661-013-1943-0,http://link.springer.com/article/10.1007/s1166...,54.6,0.023810,The Effect of Chemical Composition and Heat Tr...,"Yonezawa, Toshio",300,TEM - WBDF


We are pre-processing the collected data.

In [50]:
#First we are reducing the collected data by considering the data which is done at room temperature 
#certain column selection name["1"] if twoname[["1",""]]
data_red1 = collected_data[(collected_data["temperature"]>290) & (collected_data["temperature"]<310)]
#data_roomtemp.shape
len(data_red1)

426

Now from room temperature data we will subset the data by considering experimental measurements 

In [51]:
data_red1["experiment_type"].unique()

array(['TEM - WBDF', 'XRD', 'TEM', 'Thermodynamic Modeling',
       'Synchrotron XRD', 'TEM-WBDF', 'Neutron Diffraction',
       'Literature Review', 'Literature review'], dtype=object)

In [52]:
data_red2=data_red1[(data_red1["experiment_type"]!="Thermodynamic Modeling")]
len(data_red2)

387

In [53]:
#dropping columns DOI and URL
data_red2 = data_red2.drop(['DOI', 'url'], axis=1)
data_red2.columns.values #display of column headers 

array(['C', 'N', 'P', 'S', 'V', 'Ni', 'Nb', 'Al', 'Ti', 'Fe', 'Hf', 'Mo',
       'Mn', 'Co', 'Si', 'Cr', 'Cu', 'SFE', 'Error', 'title', 'Author',
       'temperature', 'experiment_type'], dtype=object)